<a href="https://colab.research.google.com/github/Seongbuming/data-analysis/blob/main/spam-detection/Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 데이터셋

데이터셋으로는 한국우편사업진흥원의 [스팸메일 수신차단 목록](https://www.data.go.kr/data/15090173/fileData.do#/layer_data_infomation) 공공데이터를 사용한다.

원래는 내 개인 메일함을 데이터셋으로 활용하려고 했다. 11년 동안 직접 메일함 카테고리를 분류해왔기에. 그러나 스팸 메일함은 최대 보관 기간이 30일이었다. 향후에는 직접 스팸 메일함을 수집해서 분석해보아야겠다.

In [ ]:
use_colab = True
if use_colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/MyDrive/작업/2022 데이터분석/2 Spam Detection/data'
    %ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/작업/2022 데이터분석/2 Spam Detection/data
'한국우편사업진흥원_스팸메일 수신차단 목록_20211231.csv'


In [ ]:
dataset = pd.read_csv('한국우편사업진흥원_스팸메일 수신차단 목록_20211231.csv', encoding='cp949') # extended euc-kr
print(f'데이터 행 수: {len(dataset)}')

데이터 행 수: 26651


# 데이터 확인

In [ ]:
# 필드
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   수신일자    26651 non-null  object
 1   수신시간    26651 non-null  object
 2   메일종류    26651 non-null  object
 3   메일제목    26476 non-null  object
 4   첨부      8758 non-null   object
dtypes: object(5)
memory usage: 1.0+ MB


메일의 내용은 포함되어 있지 않다.

In [ ]:
# 샘플
dataset.head()

,수신일자,수신시간,메일종류,메일제목,첨부
0,2021-06-24,18:06,스팸,FW: HSBC Remttance swift copy,WIRE COPY.html (18.9 K)
1,2021-06-24,18:14,스팸,These Revolutionary Glasses Make It Safe to Dr...,없음
2,2021-06-24,18:14,스팸,Invest in your safety and peace of mind behind...,없음
3,2021-06-24,18:15,스팸,Re: PV solar glass (AR coated) for solar modul...,없음
4,2021-06-24,18:28,스팸,Cook like a samurai with Japanese knives..,없음


In [ ]:
# 결측치
dataset.isnull().sum()

수신일자        0
수신시간        0
메일종류        0
메일제목      175
첨부      17893
dtype: int64

제목이 없는 메일이 175개 있다. 일단 제거하지 않고 추후 제목 결측 메일들에 대해 종류의 분포를 확인해볼 것이다.

In [ ]:
# 제목 중복 제거
dataset.drop_duplicates(subset='메일제목', inplace=True)
print(f'중복 제거 후 데이터 행 수: {len(dataset)}')

중복 제거 후 데이터 행 수: 7052


샘플 수가 26651개에서 7052개로 줄었다. 19599개의 중복 데이터가 제거되었다.